## Extraer datos mediante la API de inmobiliaria

Pagina donde se hizo el webscraping: https://www.argenprop.com/ 

Paises admitidos: Argentina, Chile, Uruguay, Brasil(1propiedad)

In [3]:
#Librerias
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from datetime import datetime
import pytz
from unidecode import unidecode
import re

## 1. Creamos la conexión con la API para extraer los datos

In [4]:
%%time

#url = 'http://reffindr-alb-1167121448.us-east-1.elb.amazonaws.com:4155/argenprop'
url = 'https://i004-reffindr-back-python-dev.onrender.com/argenprop'

# Parámetros de la solicitud
params = {'pais': 'argentina', 'limite': 300}
response = requests.get(url, params=params)
print(response.status_code)

200
CPU times: total: 46.9 ms
Wall time: 5min 16s


In [7]:
data = response.json()

In [8]:
df = pd.DataFrame(data)
df

,Bathrooms,Bedrooms,CountryName,Description,Environments,Latitude,Longitude,Price,Seniority,StateName,Title,img
0,2,3,Argentina,"Casa en Barrio Privado Camino Real, cuenta con...",4,"-34,48684","-58,58234",USD 2.300,27,Buenos Aires,Av. Camino Real Morón San Fernando 1500,[https://static1.sosiva451.com/08910661/ea4bac...
1,2,3,Argentina,Alquilo amplio Duplex en Florencio varela comb...,4,"-34,79154","-58,27525",$ 490.000,10,Buenos Aires,Combate De San Lorenzo 1700,[https://static1.sosiva451.com/28339561/8cca67...
2,2,4,Argentina,DUEÑO ALQUILA\r\n\n •SIN EXPENSAS\r\n •SIN GAS...,5,0.0,0.0,USD 1.100,15,Buenos Aires,casa en lujan con pileta alquiler anual,[https://static1.sosiva451.com/09149461/6abd53...
3,2,4,Argentina,Imponente casa de estilo en la esquina de Alve...,5,"-32,94819","-60,66076",$ 2.300.000,90,Santa Fe,Mendoza 2300,[https://static1.sosiva451.com/22799461/e73d3f...
4,5,5,Argentina,Exclusiva Casa en Alquiler en Puertos del Lago...,7,"-34,318497","-58,742558",USD 7.500,0,Buenos Aires,Puertos del Lago Barrio Marinas Escobar Alquiler,[https://static1.sosiva451.com/48236461/e64ab1...
...,...,...,...,...,...,...,...,...,...,...,...,...
295,4,5,Argentina,Dueño directo \n Deportiva I- L146\n\n Barrio...,6,"-37,0089","-56,80412",USD 6.500,1,Buenos Aires,Costa Esmeralda-deportivo I,[https://static1.sosiva451.com/62641321/178b35...
296,5,None,Argentina,EDIFICIO MONTEVIDEO 359\r\n Nueva Cordoba\r\n\...,None,"-31,420574","-64,191246",$ 2.000.000,0,Cordoba,Montevideo 300,[https://static1.sosiva451.com/98857951/d3b8d0...
297,3,3,Argentina,Alquiler. \n Semipiso de 4 ambientes en edific...,4,"-34,48799","-58,49957",USD 1.600,0,Buenos Aires,Vicente Fidel López 100,[https://static1.sosiva451.com/26151061/71045e...
298,5,3,Argentina,Hermosa y amplia Casa desarrollada en dos plan...,4,"-34,36595","-58,7351",USD 1.350,6,Buenos Aires,La Pista 100,[https://static1.sosiva451.com/26798261/62d8a1...


## 1.1 Creación de Tabla de States

In [9]:
data_states = {
    1: "Buenos Aires",
    2: "Catamarca",
    3: "Chaco",
    4: "Chubut",
    5: "Córdoba",
    6: "Corrientes",
    7: "Entre Ríos",
    8: "Formosa",
    9: "Jujuy",
    10: "La Pampa",
    11: "La Rioja",
    12: "Mendoza",
    13: "Misiones",
    14: "Neuquén",
    15: "Río Negro",
    16: "Salta",
    17: "San Juan",
    18: "San Luis",
    19: "Santa Cruz",
    20: "Santa Fe",
    21: "Santiago del Estero",
    22: "Tierra del Fuego",
    23: "Tucumán",
    24: "Ciudad Autónoma de Buenos Aires",
}

# Crear un DataFrame con los estados
States = pd.DataFrame(data_states.items(), columns=["Id", "StateName"])

In [10]:
States

,Id,StateName
0,1,Buenos Aires
1,2,Catamarca
2,3,Chaco
3,4,Chubut
4,5,Córdoba
5,6,Corrientes
6,7,Entre Ríos
7,8,Formosa
8,9,Jujuy
9,10,La Pampa


## 2. Transformación de datos

### 2.0 Transformación Tabla de datos de Propietarios

Copia del df original

In [ ]:
df_properties = df.copy()

Función para convertir la columna Price en moneda de pesos argentinos

In [ ]:
def convert_to_ars(price):
    exchange_rate = 1011.61 
    price = str(price)
    if 'USD' in price:
        price_numeric = float(price.replace('USD', '').replace('.', '').replace(',', '.').strip())
        return price_numeric * exchange_rate
    elif '$' in price:
        price_numeric = float(price.replace('$', '').replace('.', '').replace(',', '.').strip())
        return price_numeric
    else:
        return None

In [ ]:
df_properties['Price'] = df_properties['Price'].apply(convert_to_ars)

Arreglo de string de latitud y longitud

In [ ]:
# Reemplazar comas por puntos solo en las columnas Latitude y Longitude
df_properties['Latitude'] = df_properties['Latitude'].apply(lambda x: str(x).replace(',', '.') if isinstance(x, str) else str(x))
df_properties['Longitude'] = df_properties['Longitude'].apply(lambda x: str(x).replace(',', '.') if isinstance(x, str) else str(x))

# Convertir las columnas a float
df_properties['Latitude'] = pd.to_numeric(df_properties['Latitude'], errors='coerce')
df_properties['Longitude'] = pd.to_numeric(df_properties['Longitude'], errors='coerce')


Eliminando filas que contienen valores en blanco (' ')

In [ ]:
string_columns = df_properties.select_dtypes(include=['object', 'string'])
rows_with_spaces = string_columns.apply(lambda col: col.str.strip() == '', axis=0).any(axis=1)
df_properties = df_properties[~rows_with_spaces].reset_index(drop=True)

In [ ]:
df_properties.isnull().sum()

Bathrooms       29
Bedrooms        22
CountryName      0
Description      0
Environments    33
Latitude         0
Longitude        0
Price           10
Seniority        0
StateName        0
Title            0
img              0
dtype: int64

Eliminando filas con valores nulos

In [ ]:
df_properties = df_properties.dropna().reset_index(drop=True)

In [ ]:
df_properties.isnull().sum()

Bathrooms       0
Bedrooms        0
CountryName     0
Description     0
Environments    0
Latitude        0
Longitude       0
Price           0
Seniority       0
StateName       0
Title           0
img             0
dtype: int64

Función para obtener dirección en base a latitud y longitud

In [ ]:
geolocator = Nominatim(user_agent="geoapi_exercises")

# Función para obtener la dirección
def obtener_direccion(lat, lon):
    if lat == 0.0 and lon == 0.0:
        return None 
    try:
        location = geolocator.reverse((lat, lon))
        return location.address if location else None
    except Exception as e:
        return f"Error: {e}"

In [ ]:
df_properties['Address'] = df_properties.apply(lambda x: obtener_direccion(x['Latitude'], x['Longitude']), axis=1)

In [ ]:
df_properties.dropna(inplace=True) 
df_properties.reset_index(drop=True, inplace=True)

In [ ]:
states_list = states["StateName"].tolist()

# Función para extraer las últimas 4 partes de la dirección
def extract_relevant_address(address):
    # Dividir la dirección por comas
    parts = address.split(", ")
    # Tomar las últimas 4 partes
    return ", ".join(parts[-4:])

# Función para extraer el estado
def extract_state(relevant_address, states_list):
    for state in states_list:
        if re.search(rf'\b{re.escape(state)}\b', relevant_address):
            return state
    return None

# Aplicar las funciones al DataFrame
df_properties["StateName"] = df_properties["Address"].apply(extract_relevant_address)
df_properties["StateName"] = df_properties["StateName"].apply(lambda x: extract_state(x, states_list))

Obtener numero de calle y/o direccion

In [ ]:
df_properties['Address'] = df_properties['Address'].apply(lambda x: ', '.join(x.split(', ')[:3]))

df_properties = df_properties.drop(columns=['Latitude', 'Longitude']) # elimino las columnas latitud y longitud

In [ ]:
df_properties.isnull().sum()

Bathrooms       0
Bedrooms        0
CountryName     0
Description     0
Environments    0
Price           0
Seniority       0
StateName       0
Title           0
img             0
Address         0
dtype: int64

In [ ]:
df_properties.dropna(inplace=True) 
df_properties.reset_index(drop=True, inplace=True)

Eliminando propiedades que no tienen imagen

In [ ]:
df_properties['img'] = df_properties['img'].astype(str)
df_properties.drop(df_properties[df_properties['img'] == '[]'].index, inplace=True)

In [ ]:
df_properties.reset_index(drop=True, inplace=True)

Eliminando titulo sin sentido 

In [ ]:
df_properties.drop(df_properties[df_properties['Title'].str.contains('U\$', na=False)].index, inplace=True)

Seleccionando 200 registros aleatoriamente

In [ ]:
df_prop = df_properties.sample(n=200, random_state=100).reset_index(drop=True)

In [ ]:
df_prop.insert(0, 'Id', range(1, len(df_prop) + 1))

Agregando columnas faltantes

In [ ]:
columnas = [
    "Water", "Gas", "Surveillance", "Electricity", "Internet", 
    "Pool", "Garage", "Pets", "Grill", "Elevator", "Terrace",
    "IsHistoric", "IsWorking", "HasWarranty", "RangeSalary"
]

# Agregar columnas con valores específicos
for column in columnas:
    if column in ["Water", "Gas", "Electricity"]:
        df_prop[column] = True  # Estas columnas son siempre True
    elif column in ["Surveillance", "Pets", "Pool"]:
        df_prop[column] = np.random.choice([True, False], size=len(df_prop))
    elif column == "RangeSalary":
        # Genero los salarios con distribución sesgada a la izquierda
        salary_range = np.random.triangular(left=400000, mode=900000, right=3000000, size=len(df_prop))
        salary_range = salary_range.astype(int)  # Asegurarnos de que sean números enteros
        df_prop[column] = salary_range
    else:
        df_prop[column] = np.random.choice([True, False], size=len(df_prop), p=[0.8, 0.2]) # sesgo para que haya mas true que false

In [ ]:
df_prop.loc[:, 'CreatedAt'] = datetime.now(pytz.UTC)
df_prop['UpdatedAt'] = pd.Series([None] * len(df_prop), dtype="datetime64[ns]")
df_prop['IsDeleted'] = False

Ordenando columnas

In [ ]:
orden_columns = [
    "Id","img","IsWorking", "HasWarranty", "RangeSalary", "CountryName", "StateName", "Title", "Address", "Price", "Environments", 
    "Bathrooms", "Bedrooms", "Seniority", "Water", "Gas", "Surveillance", "Electricity", "Internet", "Pool", 
    "Garage", "Pets", "Grill", "Elevator", "Terrace", "IsHistoric", "Description", "CreatedAt", "UpdatedAt",
    "IsDeleted"
]

df_prop = df_prop[orden_columns]

Rangos Salariales

In [ ]:
ranges = [
    (300000, 600000),
    (600000, 1000000),
    (1000000, 3000000),
    (3000000, float("inf"))
]

# Función para asignar el rango
def assign_salary_range(salary):
    for i, (low, high) in enumerate(ranges, start=1):
        if low <= salary < high:
            return i
    return None

In [ ]:
df_prop["RangeSalary"] = df_prop["RangeSalary"].apply(assign_salary_range)

In [ ]:
df_prop.rename(columns={'RangeSalary': 'SalaryId'}, inplace=True)

Cambiando a int las columnas Environments, Bathrooms, Bedrooms, Seniority

In [ ]:
df_prop[['Environments', 'Bathrooms', 'Bedrooms', 'Seniority']] = df_prop[['Environments', 'Bathrooms', 'Bedrooms', 'Seniority']].astype('int64')

In [ ]:
df_prop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 30 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   Id            200 non-null    int64              
 1   img           200 non-null    object             
 2   IsWorking     200 non-null    bool               
 3   HasWarranty   200 non-null    bool               
 4   SalaryId      200 non-null    int64              
 5   CountryName   200 non-null    object             
 6   StateName     200 non-null    object             
 7   Title         200 non-null    object             
 8   Address       200 non-null    object             
 9   Price         200 non-null    float64            
 10  Environments  200 non-null    int64              
 11  Bathrooms     200 non-null    int64              
 12  Bedrooms      200 non-null    int64              
 13  Seniority     200 non-null    int64              
 14  Water     

In [ ]:
df_prop

,Id,img,IsWorking,HasWarranty,SalaryId,CountryName,StateName,Title,Address,Price,...,Garage,Pets,Grill,Elevator,Terrace,IsHistoric,Description,CreatedAt,UpdatedAt,IsDeleted
0,1,['https://static1.sosiva451.com/55412161/f9e15...,True,True,3,Argentina,Buenos Aires,Tiscornia 1000,"1043, Avenida Tiscornia, La Calabria",2529025.0,...,True,True,True,True,True,True,"Muy buena En pleno bajo de San Isidro, sobre ...",2024-12-07 04:23:14.435484+00:00,NaT,False
1,2,['https://static1.sosiva451.com/31652661/f130e...,False,True,3,Argentina,Buenos Aires,De Vicenzo Grande La Esperanza 2300,"Golf Club Villa Adelina, Manuel Alberti, Parti...",780000.0,...,True,False,True,True,False,True,"Ubicada en la zona de De Vicenzo Grande, esta ...",2024-12-07 04:23:14.435484+00:00,NaT,False
2,3,['https://static1.sosiva451.com/09087751/dc11e...,True,False,3,Argentina,Buenos Aires,Los Cardales Country Club al 100,"Ruta 4, Alto Los Cardales, Partido de Campana",910449.0,...,True,False,True,False,False,False,Casa en una plata con vista a Golf.\n living c...,2024-12-07 04:23:14.435484+00:00,NaT,False
3,4,['https://static1.sosiva451.com/42174661/7c7e6...,True,True,3,Argentina,Buenos Aires,ESPAÑA 3900,"3968, España, Loma de Roca",1350000.0,...,False,True,False,True,True,True,"Alquiler de Casa en Olivos, Vicente López\n V...",2024-12-07 04:23:14.435484+00:00,NaT,False
4,5,['https://static1.sosiva451.com/91888461/b04bf...,True,True,3,Argentina,Buenos Aires,St. Matthews Village,"Barrio Saint Matthew's Village, La Lonja, Part...",2225542.0,...,False,False,True,True,True,True,Alquiler de Casa 5 AMBIENTES en St. Mathew´s V...,2024-12-07 04:23:14.435484+00:00,NaT,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,['https://static1.sosiva451.com/01961661/ba760...,False,True,3,Argentina,Buenos Aires,Manuel Quintana 900,"938, Manuel Quintana, Quilmes Este",600000.0,...,False,False,False,True,True,False,En esta oportunidad presentamos esta casa 3 am...,2024-12-07 04:23:14.435484+00:00,NaT,False
196,197,['https://static1.sosiva451.com/87382461/f69b4...,True,False,3,Argentina,Buenos Aires,Haras María Victoria,"Barrio El Aromo, Loma Verde, Partido de Escobar",313599100.0,...,True,True,True,True,True,True,Casa en venta en Haras María Victoria.\n Actua...,2024-12-07 04:23:14.435484+00:00,NaT,False
197,198,['https://static1.sosiva451.com/66551161/d6828...,True,True,3,Argentina,Buenos Aires,San Nicolás 200,"3612, Bogotá, Floresta",1300000.0,...,False,True,False,True,True,True,SOLAMENTE PARA FINES COMERCIALES o DEPOSITO.-\...,2024-12-07 04:23:14.435484+00:00,NaT,False
198,199,['https://static1.sosiva451.com/00842661/3f046...,False,False,3,Argentina,Buenos Aires,Domingo Savio 2880/2,"Domingo Savio, Lomas de San Isidro, San Isidro",1600000.0,...,True,True,True,False,True,True,Excelente ubicación: 2 cuadras de Segundo Fern...,2024-12-07 04:23:14.435484+00:00,NaT,False


### 2.1 Transformación Tabla de datos de Usuarios

Leyendo datos fictios creados por IA

In [ ]:
csv_path = 'Data_ficticia\\Users_Ficticios_IA.csv'
df_users = pd.read_csv(csv_path, delimiter=';')

Seleccionando 400 usuarios aleatoriamente

In [ ]:
df_users = df_users.sample(n=400, random_state=5).reset_index(drop=True)

Transformación de datos

In [ ]:
# Función para quitar los dos últimos elementos
def remove_last_two_parts(address):
    parts = address.split(', ')  # Dividir por la coma y el espacio
    return ', '.join(parts[:-2])  # Reunir todas las partes excepto las dos últimas

# Aplicar la función a la columna 'Address'
df_users['Address'] = df_users['Address'].apply(remove_last_two_parts)

Creando columnas faltantes

In [ ]:
df_users.insert(0, 'Id', range(1, len(df_users) + 1))
df_users.insert(1, 'RoleName', ['Tenant' if i % 2 == 0 else 'Owner' for i in range(len(df_users))])
df_users.insert(2, 'CountryName', 'Argentina')
df_users.insert(4, 'IsCompany', df_users['RoleName'].apply(lambda x: 'False' if x == 'Owner' else ''))
df_users.insert(13, 'IsProfileComplete', 'True')
df_users.insert(15, 'CreatedAt', datetime.now(pytz.UTC))
df_users.insert(16, 'UpdatedAt', pd.Series([None] * len(df_users), dtype="datetime64[ns]"))
df_users.insert(17, 'IsDeleted', False)

Enumerando las filas tanto para Owner y Tenant

In [ ]:
df_users.insert(4, 'UserOwnerInfoId', pd.Series([None] * len(df_users), dtype="Int64"))
df_users.insert(5, 'UserTenantInfoId', pd.Series([None] * len(df_users), dtype="Int64"))

owner_counter = 1
tenant_counter = 1

for index, row in df_users.iterrows():
    if row['RoleName'] == 'Owner':
        df_users.at[index, 'UserOwnerInfoId'] = owner_counter
        owner_counter += 1
    elif row['RoleName'] == 'Tenant':
        df_users.at[index, 'UserTenantInfoId'] = tenant_counter
        tenant_counter += 1

In [ ]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   Id                 400 non-null    int64              
 1   RoleName           400 non-null    object             
 2   CountryName        400 non-null    object             
 3   StateName          400 non-null    object             
 4   UserOwnerInfoId    200 non-null    Int64              
 5   UserTenantInfoId   200 non-null    Int64              
 6   IsCompany          400 non-null    object             
 7   Name               400 non-null    object             
 8   LastName           400 non-null    object             
 9   Dni                400 non-null    int64              
 10  Phone              400 non-null    object             
 11  Address            400 non-null    object             
 12  BirthDate          400 non-null    object         

In [ ]:
df_users

,Id,RoleName,CountryName,StateName,UserOwnerInfoId,UserTenantInfoId,IsCompany,Name,LastName,Dni,Phone,Address,BirthDate,Email,Password,IsProfileComplete,GenreName,CreatedAt,UpdatedAt,IsDeleted
0,1,Tenant,Argentina,San Juan,<NA>,1,,Trinidad,Farre,92309378,+54 9 11-5516-2964,Acceso Nydia Gimenez 975,3/04/1971,trinidad.farre@gmail.com,@Farre03,True,Female,2024-12-07 04:28:17.001324+00:00,NaT,False
1,2,Owner,Argentina,Salta,1,<NA>,False,Victorino,Valentin,19496248,+54 9 21-5986-6054,Camino del Libertador 1304,20/05/1999,victorino.valentin@gmail.com,<Valentin20,True,Male,2024-12-07 04:28:17.001324+00:00,NaT,False
2,3,Tenant,Argentina,Jujuy,<NA>,2,,Rosenda,Barreda,12697037,+54 9 21-5297-7790,Pasaje de Emperatriz Ribes 56,20/01/1980,rosenda.barreda@gmail.com,@Barreda20,True,Female,2024-12-07 04:28:17.001324+00:00,NaT,False
3,4,Owner,Argentina,Salta,2,<NA>,False,Panfilo,Vergara,57455334,+54 9 21-4995-5021,Alameda de Heliodoro Hurtado 317,29/07/1952,panfilo.vergara@gmail.com,@Vergara29,True,Male,2024-12-07 04:28:17.001324+00:00,NaT,False
4,5,Tenant,Argentina,Corrientes,<NA>,3,,Lucia,Arrieta,16391990,+54 9 11-4236-3158,Ronda de Quirino Suarez 3476,8/09/1953,lucia.arrieta@gmail.com,]Arrieta08,True,Female,2024-12-07 04:28:17.001324+00:00,NaT,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,396,Owner,Argentina,Córdoba,198,<NA>,False,Pablo,Balaguer,64223341,+54 9 12-4995-3708,Calle Rivadavia 456,11/07/1974,pablo.balaguer@gmail.com,^Balaguer11,True,Male,2024-12-07 04:28:17.001324+00:00,NaT,False
396,397,Tenant,Argentina,Jujuy,<NA>,199,,David,Haro,34314725,+54 9 18-6704-7051,Boulevard Juan Perón 4826,24/08/1974,david.haro@gmail.com,)Haro24,True,Male,2024-12-07 04:28:17.001324+00:00,NaT,False
397,398,Owner,Argentina,Córdoba,199,<NA>,False,Zaira,Coloma,34874997,+54 9 17-6453-8194,Calle Rivadavia 456,20/08/1987,zaira.coloma@gmail.com,!Coloma20,True,Female,2024-12-07 04:28:17.001324+00:00,NaT,False
398,399,Tenant,Argentina,Santa Cruz,<NA>,200,,Juan Antonio,Palmer,55627846,+54 9 22-6995-7719,Camino del Libertador 346,20/07/1955,juanantonio.palmer@gmail.com,.Palmer20,True,Male,2024-12-07 04:28:17.001324+00:00,NaT,False
